In [48]:
import pandas as pd
df=pd.read_csv("/content/URL_ID.csv")

In [49]:
df.head()

,url,title,description
0,https://insights.blackcoffer.com/rising-it-cit...,NaN,We have seen a huge development and dependence...
1,https://insights.blackcoffer.com/rising-it-cit...,NaN,"Throughout history, from the industrial revolu..."
2,https://insights.blackcoffer.com/internet-dema...,NaN,Introduction\nIn the span of just a few decade...
3,https://insights.blackcoffer.com/rise-of-cyber...,NaN,"The way we live, work, and communicate has unq..."
4,https://insights.blackcoffer.com/ott-platform-...,NaN,The year 2040 is poised to witness a continued...


In [50]:
df.drop(columns=["url","title"],axis=1,inplace=True)

In [51]:
df.head()

,description
0,We have seen a huge development and dependence...
1,"Throughout history, from the industrial revolu..."
2,Introduction\nIn the span of just a few decade...
3,"The way we live, work, and communicate has unq..."
4,The year 2040 is poised to witness a continued...


In [52]:
df.isnull().sum()

description    0
dtype: int64

# Remove stopwords

In [53]:
from bs4 import BeautifulSoup
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [54]:
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remove special characters and digits
    text = re.sub(r"[^a-zA-Z]", " ", text)

    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))


    # Remove emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Convert to lowercase
    text = text.lower()

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    text = ' '.join(tokens)

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    text = ' '.join(tokens)

    return text

In [55]:
df["description"]=df['description'].apply(clean_text)

# 8	Average Word Length

In [56]:
avg=df['description'].apply(len)
avg_len=avg.mean()
print(f"Average Text Length: {avg_len}")

Average Text Length: 5085.8877551020405


#  average sentence length

In [57]:
import re
from statistics import mean
def calculate_average_sentence_length(text):
    sentences = re.split(r'[.!?]', text)  # Split sentences using ., !, or ? as delimiters
    sentences = [s.strip() for s in sentences if s.strip()]  # Remove empty strings
    words_per_sentence = [len(sentence.split()) for sentence in sentences]

    return mean(words_per_sentence)

df['avg_sentence_length'] = df['description'].apply(calculate_average_sentence_length)
df.head()

,description,avg_sentence_length
0,seen huge development dependence people techno...,624
1,throughout history industrial revolution th ce...,874
2,introduction span decade internet undergone as...,683
3,way live work communicate unquestionably chang...,667
4,year poised witness continued revolution world...,420


# 7.	PERCENTAGE OF COMPLEX WORDS

In [58]:
def is_complex(word):
    # Define your criterion for a complex word, e.g., word length
    return len(word) > 6

def calculate_percentage_complex_words(text):
    words = re.findall(r'\b\w+\b', text)  # Extract words from the text
    complex_words = [word for word in words if is_complex(word)]

    if not words:  # Handle the case where there are no words in the text
        return 0

    percentage_complex_words = (len(complex_words) / len(words)) * 100
    return percentage_complex_words

df['percentage_complex_words'] = df['description'].apply(calculate_percentage_complex_words)
df.head()

,description,avg_sentence_length,percentage_complex_words
0,seen huge development dependence people techno...,624,35.737179
1,throughout history industrial revolution th ce...,874,51.487414
2,introduction span decade internet undergone as...,683,63.689605
3,way live work communicate unquestionably chang...,667,60.119940
4,year poised witness continued revolution world...,420,53.333333


# 2	Analysis of Readability

In [59]:
def calculate_gunning_fox_index(text):
    # Calculate Average Sentence Length
    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text)
    average_sentence_length = len(words) / len(sentences)

    # Calculate Percentage of Complex words
    complex_words = [word for word in words if len(word) > 6]  # Example criterion for a complex word
    percentage_complex_words = (len(complex_words) / len(words)) * 100

    # Calculate Fog Index
    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)

    return fog_index

df["gunning_fox_index"] = df['description'].apply(calculate_gunning_fox_index)
df.head()


,description,avg_sentence_length,percentage_complex_words,gunning_fox_index
0,seen huge development dependence people techno...,624,35.737179,263.894872
1,throughout history industrial revolution th ce...,874,51.487414,370.194966
2,introduction span decade internet undergone as...,683,63.689605,298.675842
3,way live work communicate unquestionably chang...,667,60.119940,290.847976
4,year poised witness continued revolution world...,420,53.333333,189.333333


# 5	Word Count

In [60]:
def calculate_word_count(text):
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)
    cleaned_words = [word for word in words if word not in stop_words]

    # Remove punctuation
    cleaned_words = [word.translate(str.maketrans("", "", string.punctuation)) for word in cleaned_words]

    # Remove empty strings
    cleaned_words = [word for word in cleaned_words if word]

    # Count the cleaned words
    word_count = len(cleaned_words)

    return word_count


# Apply the calculate_word_count function to the 'description' column
df["word_count"] = df['description'].apply(calculate_word_count)
df.head()


,description,avg_sentence_length,percentage_complex_words,gunning_fox_index,word_count
0,seen huge development dependence people techno...,624,35.737179,263.894872,624
1,throughout history industrial revolution th ce...,874,51.487414,370.194966,874
2,introduction span decade internet undergone as...,683,63.689605,298.675842,683
3,way live work communicate unquestionably chang...,667,60.119940,290.847976,667
4,year poised witness continued revolution world...,420,53.333333,189.333333,420


# 3	Average Number of Words Per Sentence

In [61]:
def calculate_avg_words_per_sentence(text):
    # Tokenize the text into sentences and words
    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text)

    # Calculate the average number of words per sentence
    avg_words_per_sentence = len(words) / len(sentences) if len(sentences) > 0 else 0

    return avg_words_per_sentence

df["avg_words_per_sentence"] = df['description'].apply(calculate_avg_words_per_sentence)
df.head()

,description,avg_sentence_length,percentage_complex_words,gunning_fox_index,word_count,avg_words_per_sentence
0,seen huge development dependence people techno...,624,35.737179,263.894872,624,624.0
1,throughout history industrial revolution th ce...,874,51.487414,370.194966,874,874.0
2,introduction span decade internet undergone as...,683,63.689605,298.675842,683,683.0
3,way live work communicate unquestionably chang...,667,60.119940,290.847976,667,667.0
4,year poised witness continued revolution world...,420,53.333333,189.333333,420,420.0


# 7	Personal Pronouns

In [63]:
def calculate_personal_pronouns(text):
    # Define a regex pattern to find personal pronouns
    pronoun_pattern = re.compile(r'\b(?:I|we|my|ours|us)\b', flags=re.IGNORECASE)

    # Find all matches in the text
    matches = pronoun_pattern.findall(text)

    # Count the occurrences of personal pronouns
    pronoun_count = len(matches)

    return pronoun_count
df["personal_pronouns_count"] = df['description'].apply(calculate_personal_pronouns)
df.head()

,description,avg_sentence_length,percentage_complex_words,gunning_fox_index,word_count,avg_words_per_sentence,personal_pronouns_count
0,seen huge development dependence people techno...,624,35.737179,263.894872,624,624.0,0
1,throughout history industrial revolution th ce...,874,51.487414,370.194966,874,874.0,0
2,introduction span decade internet undergone as...,683,63.689605,298.675842,683,683.0,0
3,way live work communicate unquestionably chang...,667,60.119940,290.847976,667,667.0,0
4,year poised witness continued revolution world...,420,53.333333,189.333333,420,420.0,0


# 6	Syllable Count Per Word

In [66]:
def count_syllables_per_word(word):
    # Define exceptions for words ending with "es" or "ed"
    exceptions = ["es", "ed"]

    # Count the number of vowels in the word
    vowels = "aeiouy"
    syllable_count = 0
    prev_char = ''

    for char in word:
        if char in vowels and prev_char not in vowels:
            syllable_count += 1
        prev_char = char

    # Handle exceptions
    if word.endswith(tuple(exceptions)):
        syllable_count -= 1

    # Ensure the count is at least 1 for words without vowels
    syllable_count = max(syllable_count, 1)

    return syllable_count

def calculate_syllable_count(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)

    # Count the syllables for each word
    syllables_per_word = [count_syllables_per_word(word) for word in words]

    # Calculate the total syllable count
    total_syllable_count = sum(syllables_per_word)

    return total_syllable_count

df["syllable_count"] = df['description'].apply(calculate_syllable_count)
df.head()


,description,avg_sentence_length,percentage_complex_words,gunning_fox_index,word_count,avg_words_per_sentence,personal_pronouns_count,syllable_count
0,seen huge development dependence people techno...,624,35.737179,263.894872,624,624.0,0,1258
1,throughout history industrial revolution th ce...,874,51.487414,370.194966,874,874.0,0,1943
2,introduction span decade internet undergone as...,683,63.689605,298.675842,683,683.0,0,1723
3,way live work communicate unquestionably chang...,667,60.119940,290.847976,667,667.0,0,1642
4,year poised witness continued revolution world...,420,53.333333,189.333333,420,420.0,0,883


# 1	Sentimental Analysis

In [71]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Download NLTK data
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

# Define a function to get sentiment scores
def get_sentiment_score(text):
    return sia.polarity_scores(text)['compound']

# Apply the function to the 'financial_text' column
df['sentiment_score'] = df['description'].apply(get_sentiment_score)

def categorize_sentiment(score):
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the function to create a 'sentiment' column
df['sentiment'] = df['sentiment_score'].apply(categorize_sentiment)
df.head()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,description,avg_sentence_length,percentage_complex_words,gunning_fox_index,word_count,avg_words_per_sentence,personal_pronouns_count,syllable_count,sentiment_score,sentiment
0,seen huge development dependence people techno...,624,35.737179,263.894872,624,624.0,0,1258,0.9993,positive
1,throughout history industrial revolution th ce...,874,51.487414,370.194966,874,874.0,0,1943,0.9993,positive
2,introduction span decade internet undergone as...,683,63.689605,298.675842,683,683.0,0,1723,0.9963,positive
3,way live work communicate unquestionably chang...,667,60.119940,290.847976,667,667.0,0,1642,-0.9938,negative
4,year poised witness continued revolution world...,420,53.333333,189.333333,420,420.0,0,883,0.9973,positive


# 1.3	Extracting Derived variables

In [75]:
from nltk.tokenize import word_tokenize
# Load Positive and Negative words sets (you can create these sets using the previous step)
positive_words = set(["positive", "good", "excellent"])  # Replace with your positive words
negative_words = set(["negative", "bad", "poor"])  # Replace with your negative words

# Load NLTK stop words
stop_words = set(stopwords.words('english'))

def extract_derived_variables(text):
    # Tokenize the text into words
    tokens = word_tokenize(text.lower())

    # Calculate Positive Score
    positive_score = sum(1 for word in tokens if word in positive_words and word not in stop_words)

    # Calculate Negative Score
    negative_score = -sum(1 for word in tokens if word in negative_words and word not in stop_words)

    # Calculate Polarity Score
    polarity_score = (positive_score - negative_score) / (positive_score + negative_score + 1e-6)

    # Calculate Subjectivity Score
    subjectivity_score = (positive_score + negative_score) / (len(tokens) + 1e-6)

    return {
        'Positive Score': positive_score,
        'Negative Score': negative_score,
        'Polarity Score': polarity_score,
        'Subjectivity Score': subjectivity_score
    }

df['derived_variables'] = df['description'].apply(extract_derived_variables)
df.head()

,description,avg_sentence_length,percentage_complex_words,gunning_fox_index,word_count,avg_words_per_sentence,personal_pronouns_count,syllable_count,sentiment_score,sentiment,derived_variables
0,seen huge development dependence people techno...,624,35.737179,263.894872,624,624.0,0,1258,0.9993,positive,"{'Positive Score': 7, 'Negative Score': 0, 'Po..."
1,throughout history industrial revolution th ce...,874,51.487414,370.194966,874,874.0,0,1943,0.9993,positive,"{'Positive Score': 0, 'Negative Score': -1, 'P..."
2,introduction span decade internet undergone as...,683,63.689605,298.675842,683,683.0,0,1723,0.9963,positive,"{'Positive Score': 0, 'Negative Score': 0, 'Po..."
3,way live work communicate unquestionably chang...,667,60.119940,290.847976,667,667.0,0,1642,-0.9938,negative,"{'Positive Score': 0, 'Negative Score': 0, 'Po..."
4,year poised witness continued revolution world...,420,53.333333,189.333333,420,420.0,0,883,0.9973,positive,"{'Positive Score': 1, 'Negative Score': 0, 'Po..."
